Всё домашнее задание необходимо сделать ТОЛЬКО на SQL
Ссылка на документацию по SQL: https://spark.apache.org/docs/latest/sql-ref.html
Изначально предполагается что базовые таблицы уже загружены в проверочную систему в CSV формате с типами полей STRING:
 
title_basics_csv
title_principals_csv
title_crew_csv
title_episode_csv
title_ratings_csv
title_akas_csv
Name_basics_csv
 
Для преобразования STRING в другой тип, можно воспользоваться вот этой конструкцией: “CAST (column_name AS TYPE) AS column_name” на этапе инициализации:

Например:
CREATE TABLE test_table STORED AS PARQUET AS SELECT CAST(averageRating AS decimal(2,1))  AS averageRating FROM test_table_csv 

Перед этим не мешает проверить, что значения влезают в размерности этого типа
Список типов SQL: https://spark.apache.org/docs/latest/sql-ref-datatypes.html 

Во всех задачах ответом должно быть только ОДНО значение (число или строка).

Неправильный ответ - 0 баллов за задачу.
Правильный ответ на все тесты 1 балл, иначе пропорционально количеству пройденных тестов (количество тестов и какие именно заранее неизвестны).

Всего 5 задач и 5 баллов

**В режиме проверки SQL запросы будут запускаться с любыми параметрами, отличными от тех, что вы выбрали, скрипты не должны зависеть от выбранного вами параметра, а также от регистра строки. Тестирование будет проводится с любым значением параметров, в том числе с отсутвующим в датасете, запрос должен выводить правильный результат!**

SQL запрос в ответе всегда должен быть один и параметризирован. Если необходимо пользуйтесь конструкцией WITH name AS () -  пример ниже

После завершения работы нажимайте в меню File/Download/Download ipnb, скачивайте файл и присылайте почтой на ilya+hse@aniskovets.com


In [1]:
!pip install pyspark==3.2.1

!curl -O https://mars.ru77.ru/data/title.basics.tsv.gz
!curl -O https://mars.ru77.ru/data/title.crew.tsv.gz
!curl -O https://mars.ru77.ru/data/title.episode.tsv.gz
!curl -O https://mars.ru77.ru/data/title.principals.tsv.gz
!curl -O https://mars.ru77.ru/data/title.ratings.tsv.gz
!curl -O https://mars.ru77.ru/data/title.akas.tsv.gz
!curl -O https://mars.ru77.ru/data/name.basics.tsv.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 29 kB/s 
     |████████████████████████████████| 198 kB 60.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=1a7bb1ccf7a63c1842320c56fd67181aabe8b72de9df838a08e253497b855a68
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  132M  100  132M    0     0  12.5M      0  0:00:10  0:00:10 --:--:-- 17.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 51.7M  100 51.7M    0     0  9030k      0  0:00:05  0:00:05 --:--:-- 12.2M
  % Total    % Re

In [2]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf, SparkContext
 
spark = SparkSession.builder.master("local[2]").config("spark.driver.memory", "8g").appName("aig").enableHiveSupport().getOrCreate()
sql = spark.sql

title_basics_csv = spark.read.csv("title.basics.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_basics_csv.createOrReplaceTempView("title_basics_csv")
 
title_principals_csv = spark.read.csv("title.principals.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_principals_csv.createOrReplaceTempView("title_principals_csv")
 
title_crew_csv = spark.read.csv("title.crew.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_crew_csv.createOrReplaceTempView("title_crew_csv")
 
title_episode_csv = spark.read.csv("title.episode.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_episode_csv.createOrReplaceTempView("title_episode_csv")
 
title_ratings_csv = spark.read.csv("title.ratings.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_ratings_csv.createOrReplaceTempView("title_ratings_csv")
 
title_akas_csv = spark.read.csv("title.akas.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_akas_csv.createOrReplaceTempView("title_akas_csv")
 
name_basics_csv = spark.read.csv("name.basics.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
name_basics_csv.createOrReplaceTempView("name_basics_csv")

In [3]:
# Initialization
_ = sql("""DROP TABLE IF EXISTS title_basics""").collect()
_ = sql("""CREATE TABLE title_basics STORED AS PARQUET SELECT * FROM title_basics_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS title_principals""").collect()
_ = sql("""CREATE TABLE title_principals STORED AS PARQUET SELECT * FROM title_principals_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS title_crew""").collect()
_ = sql("""CREATE TABLE title_crew STORED AS PARQUET SELECT * FROM title_crew_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS title_episode""").collect()
_ = sql("""CREATE TABLE title_episode STORED AS PARQUET SELECT * FROM title_episode_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS title_ratings""").collect()
_ = sql("""CREATE TABLE title_ratings STORED AS PARQUET SELECT * FROM title_ratings_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS title_akas""").collect()
_ = sql("""CREATE TABLE title_akas STORED AS PARQUET SELECT * FROM title_akas_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS name_basics""").collect()
_ = sql("""CREATE TABLE name_basics STORED AS PARQUET SELECT * FROM name_basics_csv""").collect()

**Жанровые фильмы.** Посчитать общее количество фильмов определенного жанра

In [4]:
param1 = 'news' # жанр

sql=f"""
  WITH splitted_table AS 
  (
  SELECT tconst, explode(split(lower(genres), ',')) genre
  FROM title_basics
  )
  SELECT Count(*) as Answer
  FROM splitted_table
  WHERE genre = '{param1}'
"""
spark.sql(sql).show(50, True)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

**Хорошие жанровые фильмы.** Для определенного жанра (param1). Посчитать количество фильмов этого жанра (title_basics), у которых средний рейтинг пользователей больше или равен param2 (title_ratings:averageRating)

In [25]:
param1 = 'news' # жанр
param2 = 5.3 #  средний рейтинг

sql = f"""
  WITH splitted_table AS (
  SELECT tconst, explode(split(lower(genres), ',')) genre
  FROM title_basics
  )
  SELECT Count(DISTINCT splitted_table.tconst) AS Answer
  FROM splitted_table INNER JOIN title_ratings
    ON splitted_table.tconst = title_ratings.tconst
  WHERE genre = '{param1}' AND CAST(averageRating AS Double) >= '{param2}'
"""

spark.sql(sql).show(50, True)

+------+
|Answer|
+------+
| 13106|
+------+



**И швец, и жнец**. Посчитать количество фильмов (title_basics) определенного жанра (param1) у которых режиссер был еще и сценаристом (title_crew) (или одним из сценаристов и режиссеров одновременно)

In [ ]:
param1 = 'news' #тут любой жанр
sql = f"""
  WITH directors_splitted AS (
    SELECT tconst, explode(split(lower(directors), ',')) directors
    FROM title_crew
  ),
  writers_splitted AS (
    SELECT tconst, explode(split(lower(writers), ',')) writers
    FROM title_crew
  ),
  genre_splitted AS (
    SELECT tconst, explode(split(lower(genres), ',')) genre
    FROM title_basics
  )
  SELECT Count(DISTINCT genre_splitted.tconst) AS Answer
  FROM directors_splitted INNER JOIN writers_splitted
      ON directors_splitted.tconst = writers_splitted.tconst
        INNER JOIN genre_splitted 
      ON writers_splitted.tconst = genre_splitted.tconst
  WHERE directors = writers AND genre = '{param1}'
"""

spark.sql(sql).show(50, True)

**Любимчики режиссера.** Посчитать количество любимчиков режиссера. Любимчик режиссера (param1)  (title_crews:directors), это actress/actor (title_principals:category), который снимался в фильмах этого режиссера   >= param2 раз.

In [ ]:
param1 = 'nm0085156' # идентификатор персоны
param2 = 2 # количество фильмов для определения любимчиков >=

sql = f"""
    WITH movies as(
        SELECT title_2.nconst
        FROM title_principals title_1 INNER JOIN title_principals title_2
             ON title_1.tconst = title_2.tconst
        WHERE title_1.nconst = '{param1}' AND title_2.category in ('actress', 'actor')
        GROUP BY 1
        HAVING title_2.nconst <> '{param1}' AND count(title_1.tconst) >= {param2}
    )
    SELECT count(nconst) as Answer
    FROM movies
    """

spark.sql(sql).show(50, True)

**Займись своим делом, Bro.**
 Посчитать количество актеров и актрис (name_basics:primaryProfession) у которых режиссер (director) не является primaryProfession, а средний рейтинг фильмов в которых они все-таки были режиссерами меньше чем {param1}

In [ ]:
param1 = 5.0 # средний рейтинг фильмов актеров и актрис, которые думают, что они могут фильмы снимать.

sql = f"""
      WITH concatenation AS
      (
        SELECT nconst, concat(',', primaryProfession,',') as concated_prof
      from name_basics
      ),
      without_diploma AS
      (
        SELECT nconst, concated_prof
      FROM concatenation
      WHERE (concated_prof LIKE '%,actor,%' OR concated_prof LIKE '%,actress,%') 
      AND (concated_prof NOT LIKE '%,director,%')
      ),
      ratings AS
      (
      SELECT title_principals.tconst as tconst, title_principals.nconst as nconst,
      title_principals.category as directors,
      title_ratings.averageRating as averageRating
      FROM title_principals
      LEFT OUTER JOIN title_ratings ON title_ratings.tconst = title_principals.tconst
      WHERE title_principals.category = 'director'
      ),
      final_table AS
      (
      SELECT ratings.tconst as tconst, ratings.nconst as nconst, ratings.directors as directors,
      ratings.averageRating as averageRating, without_diploma.nconst as actors
      FROM ratings 
      LEFT OUTER JOIN without_diploma ON without_diploma.nconst = ratings.nconst
      ),
      aggregated_avg AS
      (
        SELECT actors as actors, avg(averageRating) as avg
      FROM final_table
      GROUP BY 1
      HAVING actors IS NOT NULL
      )
      SELECT COUNT(DISTINCT actors) as cnt
      FROM aggregated_avg
      WHERE avg < {param1}
"""

spark.sql(sql).show(50, True)

In [12]:
param1 = 5.0

sql = f"""
    SELECT Count(Distinct nconst)
    FROM name_basics
"""

spark.sql(sql).show(30, True)

+----------------------+
|count(DISTINCT nconst)|
+----------------------+
|              10871880|
+----------------------+



In [18]:
sql = f"""
    SELECT Count(*)
    FROM title_principals INNER JOIN title_ratings 
         ON title_ratings.tconst = title_principals.tconst
    WHERE title_principals.category = 'director'
"""
spark.sql(sql).show(30, True)

+--------+
|count(1)|
+--------+
|  978544|
+--------+



In [9]:
param1 = 5.0

sql = f"""
    WITH splitted_directors AS (
       SELECT tconst, explode(split(directors, ',')) director
       FROM title_crew
    ),
    bad_directors AS (
      SELECT director
      FROM title_ratings INNER JOIN splitted_directors
          ON title_ratings.tconst = splitted_directors.tconst
      GROUP BY director
      HAVING Avg(CAST(averageRating AS Double)) < '{param1}'
    ),
    splitted_professions AS (
      SELECT nconst, explode(split(primaryProfession, ',')) primaryProfession
      FROM name_basics 
    ),
    filtered_nconsts AS (
      SELECT nconst 
      FROM splitted_professions
      WHERE primaryProfession IN ('actor', 'actress')
      EXCEPT 
      SELECT nconst
      FROM splitted_professions
      WHERE primaryProfession = 'director'
    ),
    joined_badDirs_filteredNconsts AS (
      SELECT *
      FROM bad_directors INNER JOIN filtered_nconsts
           ON bad_directors.director = filtered_nconsts.nconst
    )
    SELECT Count(*) as Answer
    FROM joined_badDirs_filteredNconsts
"""

spark.sql(sql).show(30, True)

+------+
|Answer|
+------+
|  1620|
+------+

